# TMDb

## Inhaltsverzeichnis
- [Vorbereitungen](#Vorbereitungen)
- [Abstand der Veröffentlichung](#Abstand-der-Veröffentlichung)
- [Veröffentlichung der Serien](#Veröffentlichung-der-Serien)

## Vorbereitungen

TMDb API Key:

In [ ]:
tmdb_apikey = '78f6ddbaaf328e1f969c219240da9025'

Import der benötigten Module:

In [ ]:
import requests
import json

from pathlib import Path
from datetime import datetime, timedelta, date

import pandas as pd
import numpy as np
import plotly.express as px

Anlegen des Download Ordners:

In [ ]:
download_folder = Path('download')
download_folder.mkdir(exist_ok=True)

Laden des Datensatzes:

In [ ]:
disney_plus_titles = pd.read_csv('disney_plus_titles.csv', parse_dates=['date_added'])

Entfernen der Einträge, zu denen keine Informationen bei TMDb existieren:

In [ ]:
disney_plus_titles = disney_plus_titles.dropna(axis=0, subset='tmdb_ref')

Korrektur der Spalte `type` anhand der TMDb Ergebnisse:

In [ ]:
disney_plus_titles['type'] = disney_plus_titles['tmdb_ref'].str.split('/').str[1].replace({
    'movie': 'Movie',
    'tv': 'TV Show'
})
disney_plus_titles.head(5)
# .sample(n=5, random_state=8)

## Abstand der Veröffentlichung
**Verringert sich der Abstand zwischen Veröffentlichungen im Kino und im Streaming?**

Zur Beantwortung dieser Frage benötigen wir zu jedem Film die Angabe, wann er im Kino veröffentlicht wurde. Zunächst erfolgt nun die Auswahl der Filme bzw. das Entfernen der Serien aus der Liste aller Titel. Dazu werden Spalten entfernt, die für die weitere Betrachtung nicht notwendig sind und lediglich die Ausgabe vergrößern.

In [ ]:
disney_plus_movies = disney_plus_titles[disney_plus_titles['type'] == 'Movie']
disney_plus_movies = disney_plus_movies.drop(['director', 'cast', 'country', 'rating',
                                              'listed_in', 'duration', 'description'], axis=1)
# disney_plus_movies

Anschließend werden alle Dateien heruntergeladen, die nicht bereits im Download-Ordner enthalten sind.

In [ ]:
for _, row in disney_plus_movies.iterrows():
    # Variablen vorbereiten
    tmdb_ref = row['tmdb_ref']
    filename = tmdb_ref[1:].replace('/', '_')

    target = download_folder.joinpath(f'{filename}.json')

    # überspringen, falls Datei existiert und Daten enthält
    if target.exists() and target.stat().st_size > 200:
        continue

    # Anfrage an TMDb senden
    print('get', tmdb_ref)

    r = requests.get(f'https://api.themoviedb.org/3{tmdb_ref}?api_key={tmdb_apikey}')
    assert r.status_code == 200

    # JSON parsen und in Datei speichern
    o = r.json()
    with open(target, 'w', encoding='utf-8') as file:
        json.dump(o, file, indent=4)

Darauf werden alle diese Dateien eingelesen und die Veröffentlichungszeitpunkte und weitere Daten im DataFrame abgelegt.

In [ ]:
def load(tmdb_ref: str, field: str):
    filename = tmdb_ref[1:].replace('/', '_')
    source = download_folder.joinpath(f'{filename}.json')

    with open(source, 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data[field]

disney_plus_movies['release_date'] = pd.to_datetime(disney_plus_movies['tmdb_ref'].map(lambda tr: load(tr, 'release_date')))
disney_plus_movies.head(5)

Filme, die vor der Veröffentlichung von Disney+ am 12.11.2019 erschienen, sind nicht repräsentativ. Sie werden entfernt.

In [ ]:
disney_plus_movies = disney_plus_movies[disney_plus_movies['release_date'] > '2019-11-12'].copy()
# disney_plus_movies

Danach wird eine neue Spalte eingeführt, welche die Differenz zwischen Veröffentlichung und Aufnahme in den Online-Katalog in Tagen enthält.

In [ ]:
disney_plus_movies['release_diff'] = (disney_plus_movies['date_added'] - disney_plus_movies['release_date'])
disney_plus_movies['release_diff'] = disney_plus_movies['release_diff'].astype(np.int64) / (24 * 60 * 60 * 1000 * 1000 * 1000)
disney_plus_movies.tail(5)

Anscheinend enthält der Katalog einige Filme, die entweder falsch zugeordnet oder vor der eigentlichen Veröffentlichung aufgenommen wurden. Auch diese werden entfernt.

In [ ]:
disney_plus_movies = disney_plus_movies[disney_plus_movies['release_diff'] >= 0].copy()

Es werden weitere Daten aus den heruntergeladenenen Details geladen.

In [ ]:
for prop in 'budget', 'vote_average', 'popularity':
    disney_plus_movies[prop] = disney_plus_movies['tmdb_ref'].map(lambda tr: load(tr, prop)).replace(0, np.nan)

disney_plus_movies

Da lediglich $151$ Filme für mehrere Jahre verbleiben, wird die Veröffentlichung im Katalog auf den Monat reduziert.

In [ ]:
disney_plus_movies['month_added'] = pd.to_datetime(disney_plus_movies['date_added'].astype(str).str[:7])

Zuletzt wird alles als Scatter-Plot mit Trendlinie dargestellt. Zum Einfärben der einzelnen Punkte werden die zusätzlich geladenen Daten verwendet.

In [ ]:
def plot(color, size):
    fig = px.scatter(disney_plus_movies,
                     x='month_added', y='release_diff', trendline='ols',
                     hover_name='title',
                     title='Differenz zwischen Veröffentlichung und Aufnahme in den Katalog',
                     labels={
                         'month_added': 'Aufnahme in den Katalog',
                         'release_diff': 'Differenz in Tagen',
                         'budget': 'Budget',
                         'vote_average': 'Wertung',
                         'popularity': 'Popularität'
                     },
                     color=color,
                     size=size)
    fig.data[1].update(line_color='red')

    return fig

In [ ]:
plot('budget', 'popularity')

In [ ]:
plot('vote_average', 'popularity')

Die Differenz zwischen der Veröffentlichung im Kino und im Streaming scheint seit der Veröffentlichung von Disney+ leicht zu steigen. Eine Kategorisierung nach Budget, Zuschauerwertung und Popularität ergibt keine erkennbaren Muster hinsichtlich des Veröffentlichungszeitpunkts.

## Veröffentlichung der Serien
**Werden Serien so veröffentlicht, dass Kunden zu einem möglichst langen Beibehalten ihres Abonnements angehalten sind?**

Zur Beantwortung der Frage benötigen wir dieses Mal detaillierte Informationen über die Serien. Filme werden herausgefiltert und Spalten entfernt, die für die weitere Betrachtung nicht notwendig sind.

In [ ]:
disney_plus_shows = disney_plus_titles[disney_plus_titles['type'] == 'TV Show']
disney_plus_shows = disney_plus_shows.drop(['director', 'cast', 'country', 'rating',
                                              'listed_in', 'duration', 'description',
                                              'date_added', 'release_year'], axis=1)
# disney_plus_shows

Anschließend werden die Daten von TMDb heruntergeladen.

In [ ]:
def load_season(tmdb_ref: str, season_number: int):
    # Anfrage senden
    r = requests.get(f'https://api.themoviedb.org/3{tmdb_ref}/season/{season_number}?api_key={tmdb_apikey}')
    assert r.status_code == 200

    # parsen
    o = r.json()

    # zurückgeben
    return o


def load_show(tmdb_ref: str):
    # Anfrage senden
    r = requests.get(f'https://api.themoviedb.org/3{tmdb_ref}?api_key={tmdb_apikey}')
    assert r.status_code == 200

    # parsen
    o = r.json()

    # Details zu jeder Staffel abfragen
    o['seasons_details'] = [load_season(tmdb_ref, s['season_number'])
                            for s in o['seasons']]

    # zurückgeben
    return o


for _, row in disney_plus_shows.iterrows():
    # Variablen vorbereiten
    tmdb_ref = row['tmdb_ref']
    filename = tmdb_ref[1:].replace('/', '_')

    target = download_folder.joinpath(f'{filename}.json')

    # überspringen, falls Datei existiert und Daten enthält
    if target.exists() and target.stat().st_size > 200:
        continue

    # Anfrage an TMDb senden
    print('get', tmdb_ref)
    o = load_show(tmdb_ref)

    # in Dateisystem speichern
    with open(target, 'w', encoding='utf-8') as file:
        json.dump(o, file, indent=4)

Darauf werden alle Episoden in ein DataFrame übertragen.

In [ ]:
def load():
    download_folder = Path('download')
    for file in download_folder.iterdir():
        if not file.name.startswith('tv_'):
            continue

        with open(file, 'r', encoding='utf-8') as handle:
            data = json.load(handle)

        for season, season_detail in zip(data['seasons'], data['seasons_details']):
            for episode in season_detail['episodes']:
                s = season['season_number']
                e = episode['episode_number']

                if episode['air_date'] is not None:
                    yield data['name'], f'S{s:02}E{e:02}', datetime.strptime(episode['air_date'], '%Y-%m-%d')

names, episodes, releases = zip(*load())
disney_plus_episodes = pd.DataFrame({
    'name': names,
    'episode': episodes,
    'release': releases
})

disney_plus_episodes

Erneut werden alle Episoden vor dem Erscheinen von Disney+ entfernt. Den restlichen Folgen wird eine Spalte hinzugefügt, die die Veröffentlichungswoche enthält.

In [ ]:
def to_week_start(x):
    year, week, day = x.isocalendar()
    return date.fromisocalendar(year, week, 1)

disney_plus_weekly = disney_plus_episodes[disney_plus_episodes['release'] > '2019-11-12'].copy()
disney_plus_weekly['release_week'] = disney_plus_weekly['release'].map(to_week_start)
disney_plus_weekly

Zuletzt wird nach der entstandenen Spalte `release_week` gruppiert und gezählt.

In [ ]:
disney_plus_weekly_episodes = disney_plus_weekly.groupby('release_week')['episode'].count().reset_index()
disney_plus_weekly_episodes

Die grafische Darstellung zeigt die veröffentlichten Episoden pro Woche. Rot markiert sind jeweils die Jahreswechsel.

In [ ]:
fig = px.bar(disney_plus_weekly_episodes,
             x='release_week', y='episode',
             hover_name='release_week',
             title='Anzahl der Veröffentlichungen pro Woche',
             labels={
                 'release_week': 'Woche',
                 'episode': 'Anzahl an Veröffentlichungen',
             })

fig.add_vline(x='2020-01-01', line_width=1, line_color='red')
fig.add_vline(x='2021-01-01', line_width=1, line_color='red')
fig.add_vline(x='2022-01-01', line_width=1, line_color='red')

fig

Es bleibt festzuhalten, dass es kleinere Schwankungen innerhalb jedes Jahres gibt, so zum Beispiel um die Weihnachtszeit und den Jahreswechsel. Im Rest des Jahres scheinen Serien gleichmäßig veröffentlicht zu werden - möglicherweise auch, um Kunden durch steige Veröffentlichungen zur Beibehaltung ihres Abonnements zu überreden.

Weiterhin scheint interessant, dass im Laufe des Jahres 2022 bisher weniger Episoden veröffentlicht wurden als in den Jahren zuvor.